## Compass から勉強会名と参加率を拾ってみる

まず何がしたかったかというと、勉強会のタイトルを形態素解析して、その募集人数と参加者（＋参加待ち）の統計とったら、勉強会界隈のトレンドが追いやすくなるんじゃね？と思ったのが始まり。  
で、その為の第一歩として、Compass から勉強会情報を拾ってくる処理をまずは考えた。

### まずは requests でAPIを叩いてみる

COMPASS は [API仕様](https://connpass.com/about/api/) が公開されているので、そこから拾ってみる。

In [1]:
import requests

パラメータ組み立てて

In [2]:
compass_rest_api = 'https://connpass.com/api/v1/event/'
params = {"ym":"201904", "count":100, "order":3,"start":1}  # 4 月、100 件（max）、着信順、先頭から

query = '&'.join([f'{key}={params[key]}' for key in params.keys()])

投げてみる。

In [3]:
r_get = requests.get(compass_rest_api + '?' + query)

In [4]:
r_get.status_code

200

In [5]:
r_get.json()

{'results_returned': 100,
 'events': [{'event_url': 'https://supporterz-seminar.connpass.com/event/125251/',
   'event_type': 'advertisement',
   'owner_nickname': 'HiroakiKaede',
   'series': {'url': 'https://supporterz-seminar.connpass.com/',
    'id': 3032,
    'title': 'サポーターズCoLab'},
   'updated_at': '2019-03-23T14:05:50+09:00',
   'lat': '35.657336100000',
   'started_at': '2019-04-01T19:30:00+09:00',
   'hash_tag': 'spzcolab',
   'title': '初心者向けのディープラーニング講座【サポーターズCoLab勉強会】',
   'event_id': 125251,
   'lon': '139.699492800000',
   'waiting': 0,
   'limit': 20,
   'owner_id': 104832,
   'owner_display_name': 'HiroakiKaede',
   'description': '<h1>イベント申込方法</h1>\n<p><a href="https://supporterzcolab.com/event/789/" rel="nofollow"><img src="https://supporterzcolab.com/static/img/event/btn_apply_event_on_connpass_event_detail.png"></a></p>\n<p><img src="https://pages.supporterz.jp/rs/590-VZT-927/images/%E8%B6%85%E5%88%9D%E5%BF%83%E8%80%85.png" width="200px"></p>\n<p>※知識なしでもOK</p>\n<h1>

一度に 100 件縛りがあるので、そこだけが問題だろうか？  
とりあえず 100 件でリクエストしてみて、100 件未満になるまでループしてリクエストしてみる。

### 実際に勉強会データ（名前と参加者数）を拾ってみた

尚、やってみてわかったのだけど、ソート順がデフォルト以外だと、タイムアウトでコケるようだ。  
多分インデックスが不足してるか何かしてるんだろうなーと思いつつ。  
（いや、それ以外はそこそこ高速で返ってくる事を考えれば、RDB じゃなくて Mongo とか Cassandra とか使ってそう）

In [6]:
class ClassRoom:
    def __init__(self, title, limit, accepted, waiting):
        def or_zero(v):
            if isinstance(v, int):
                return v
            return 0
        self.title = title
        self.limit = or_zero(limit)
        self.joins = or_zero(accepted) + or_zero(waiting)
        self.score = self.joins / self.limit if self.limit != 0 else 1
        
    def __str__(self):
        return f'ClassRoom({self.title}, {self.limit}, {self.joins}, {self.score})'

def convert(event):
    return ClassRoom(event['title'], event['limit'], event['accepted'], event['waiting'])


def request_to_compass(start):
    params = {"ym":"201904", "count":WANT_EVENT, "order":1,"start":start}
    query = '&'.join([f'{key}={params[key]}' for key in params.keys()])
    return requests.get(compass_rest_api + '?' + query)


WANT_EVENT = 100
getted = WANT_EVENT
current_start = 1
class_rooms = []

while getted == WANT_EVENT:
    r_get = request_to_compass(current_start)
    respond_json = r_get.json()

    getted = respond_json['results_returned']
    current_start += WANT_EVENT
    print(f'{getted}, {current_start}')

    for ev in respond_json['events']:
        class_rooms.append(convert(ev))

# least 100
r_get = r_get = request_to_compass(current_start)
respond_json = r_get.json()
for ev in respond_json['events']:
    class_rooms.append(convert(ev))


100, 101
100, 201
100, 301
100, 401
100, 501
100, 601
86, 701


In [7]:
[str(s) for s in class_rooms]

['ClassRoom(初心者向けのディープラーニング講座【サポーターズCoLab勉強会】, 20, 0, 0.0)',
 'ClassRoom(赤坂 Masonite Python 勉強会, 15, 0, 0.0)',
 'ClassRoom(Docker+Python+Django\u3000WEB開発ハンズオン, 12, 16, 1.3333333333333333)',
 'ClassRoom([再演] 外国人との英語での働き方講座 (講師：ロッシェルさん, ちょまど, 他), 220, 0, 0.0)',
 'ClassRoom(技術同人誌を書こう【サポーターズCoLab勉強会】, 20, 0, 0.0)',
 'ClassRoom(技術同人誌を書こう【サポーターズCoLab勉強会】, 20, 0, 0.0)',
 'ClassRoom(Unagi.py 勉強会16枚目～Pythonでアルゴリズムの基本、他～, 20, 5, 0.25)',
 'ClassRoom(#ProLabo(プロラボ)渋谷もくもく会（#駆け出しエンジニア or 初心者歓迎web系)#4, 100, 59, 0.59)',
 'ClassRoom(定例モクモク勉強会, 5, 0, 0.0)',
 'ClassRoom(超入門・サーバー構築勉強会 #037 in 横浜, 8, 0, 0.0)',
 'ClassRoom(try! Swift報告会 + Swift 5勉強会【iPhone Dev Sapporo番外編】, 20, 3, 0.15)',
 'ClassRoom(MINI Hardening Project #3.2@メルカリ, 20, 0, 0.0)',
 'ClassRoom(DjangoCongress JP 2019 4月作業日, 10, 2, 0.2)',
 'ClassRoom(Webデザイン・Web制作に関する勉強会 #5, 20, 11, 0.55)',
 'ClassRoom(DevRel/Beginners #4, 14, 9, 0.6428571428571429)',
 'ClassRoom(Ansibleもくもく会 （サーバ編）2019.04 in 甲府！, 32, 1, 0.03125)',
 'ClassRoom(PWA Night vol.3 